import libraries

In [1]:
import numpy as np
import os
import glob
import pandas as pd
import PIL
from PIL import Image

# images are save in Google Drive

In [2]:
# store the images path in variable images_dir
images_dir='/content/drive/MyDrive/Colab Notebooks/processed_data/origin'

In [4]:
# create list name with images  and store all images in this list using glob and os

images = list(glob.glob(os.path.join(images_dir, '*.jpg')))
# showing image using PIL.image
PIL.Image.open(str(images[4]))

IndexError: ignored

In [5]:
# create data frame for label.list features

labels_file = "/content/drive/MyDrive/data/label/label.lst"
df = pd.read_csv(labels_file, header=None, delimiter=" ", names=['image_name', 'face_id_in_image',
                                                                 'face_box_top', 'face_box_left',
                                                                 'face_box_right','face_box_bottom',
                                                                 'face_box_cofidence', 'expression_label'])

In [6]:
df.head()

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0


In [ ]:
# crop the images using df filters and cv2 save in another  folder

# import cv2
# from tqdm import tqdm

# for index,row in tqdm(df.iterrows()):
#   imges=row['image_name']
#   extrated_images='/content/drive/MyDrive/Colab Notebooks/processed_data/origin'
#   images_path=os.path.join(extrated_images,imges)
#   image=cv2.imread(images_path)
#   if image is not None:
#     face_top=row['face_box_top']
#     face_left=row['face_box_left']
#     face_right=row['face_box_right']
#     face_bottom=row['face_box_bottom']
#     # crop the face from image
#     face=image[face_top:face_bottom,face_left:face_right]
#     #create folder for crop images
#     folder_path='/content/drive/MyDrive/Colab Notebooks/crop_images'
#     os.makedirs(folder_path,exist_ok=True)
#     crop_images_path = os.path.join(folder_path, imges)
#     cv2.imwrite(crop_images_path, face)

In [ ]:
# label names
labels=["angry","disgust","fear","happy", "sad","surprise","neutral"]

In [ ]:
# creating sub-folders for label names

# labels_folder='/content/drive/MyDrive/Colab Notebooks/processed_data/label_imgs'
# for lbl in labels:
#   os.makedirs(os.path.join(labels_folder,lbl),exist_ok=True)

In [ ]:
# create dictionary for label
mapper={
0:"angry",
1:"disgust",
2:"fear",
3:"happy",
4:"sad",
5:"surprise",
6:"neutral"}

In [8]:
# store path of crop images and img_labels in variables
images_path = '/content/drive/MyDrive/Colab Notebooks/crop_images'
labels_path = '/content/drive/MyDrive/Colab Notebooks/crop_img_labels'

In [ ]:
# copy crop images into sub-folders of labels with shutil library

# import shutil
# from tqdm import tqdm

# for label in tqdm(df.expression_label.unique()):
#     new_df = df[df.expression_label == label]
#     emotion = mapper[label]

#     for image_name in new_df.image_name:
#         img_path = os.path.join(images_path, image_name)
#         if os.path.exists(img_path):
#             dst_path = os.path.join(labels_path, emotion)
#             shutil.copy(img_path, dst_path)

In [9]:
# check how many images are in sub-folders

for folder in os.listdir(labels_path):
  files=glob.glob(pathname=str(labels_path + '/' + folder +'/*.jpg'))
  print(f'{len(files)} in folder {folder}')

2245 in folder angry
2021 in folder disgust
538 in folder fear
2638 in folder sad
15550 in folder happy
14980 in folder neutral
3395 in folder surprise


Import necessary libraries

In [11]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


 Data Preprocessing and Augmentation


In [12]:
 train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/processed_data/split folder/train',
    target_size=(150, 150),  # Adjust to the desired input size
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/processed_data/split folder/val',
    target_size=(150, 150),  # Adjust to the desired input size
    batch_size=32,
    class_mode='categorical'
)


Found 33092 images belonging to 7 classes.
Found 9421 images belonging to 7 classes.


Build the CNN model

In [10]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2),strides=1,padding='valid'))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2),strides=1,padding='valid'))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2),strides=1,padding='valid'))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


NameError: ignored

 Train the model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5
)


Evaluation and Confusion Matrix

In [ ]:
# Get predictions on validation data
y_pred = model.predict(validation_generator)

# Convert one-hot encoded vectors back to labels
y_true = np.argmax(validation_generator.classes, axis=1)
y_pred = np.argmax(y_pred, axis=1)

# Print classification report
print(classification_report(y_true, y_pred))

# Calculate and plot confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print(conf_matrix)

# Plot accuracy and loss curves
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()
